In [2]:
import time
from pathlib import Path

import pandas as pd
from tqdm import tqdm
import imdb
from imdb import Cinemagoer
from deepface import DeepFace

2023-10-07 20:09:52.099746: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-07 20:09:52.747653: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/amos/local/anaconda/envs/homeland/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda-11.2/lib64
2023-10-07 20:09:52.747725: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PAT

In [9]:
ia = Cinemagoer()

In [3]:
def cast_from_series(df,
                     count=0):
    cast_ids = []
    for cast in episode_df['cast']:
        cast_ids.extend(cast.split(','))
    cast_ids = list(set(cast_ids))

    data = []
    for cast_id in cast_ids:
        temp = episode_df[episode_df['cast'].str.contains(cast_id)]
        datum = {'cast_id': cast_id,
                 'count': temp.shape[0]}
        data.append(datum)
    cast_count = pd.DataFrame(data)
    cast_count = cast_count.sort_values(by='count')
    cast_df = cast_count[cast_count['count'] >= count]
    return cast_df['cast_id'].tolist()

In [4]:
def get_cast_member(cast_member):
    person = ia.get_person(cast_member.personID)
    if len(cast_member.currentRole) > 1:
        r = cast_member.currentRole[0]
    else:
        r = cast_member.currentRole
    notes = r.notes
    if notes == '(uncredited)':
        return None
    else:
        try:
            headshot = person['headshot']
        except KeyError:
            return None
        return {'name': r.data['name'],
                'personID': cast_member.personID}

In [5]:
def cast_from_episode(episode_id,
                      cast=None):
    series = ia.get_movie(episode_id)
    
    data = []
    cast_members = series.data['cast'] if cast is None else [x for x in series.data['cast'] if x.personID in cast]
    for cast_member in tqdm(cast_members, desc='Getting cast'):
        cnt = 0
        while cnt < 5:
            try:
                datum = get_cast_member(cast_member)
                if datum is not None:
                    data.append(datum)
                break
            except Exception as e:
                print(e)
                time.sleep(1)
                cnt += 1
                continue
    return data

In [6]:
episode_df = pd.read_csv('./data/episodes.csv', index_col=0)
episode_df = episode_df[episode_df['series_id'] == int('0412142')]
episode_df.head()

,series_id,episode_id,title,year,season,episode,cast
96,412142,606035,Pilot,2004,1,1,"0491402,0249046,0004898,0000494,0607185,081798..."
97,412142,606034,Paternity,2004,1,2,"0491402,0249046,0004898,0000494,0607185,081798..."
98,412142,606033,Occam's Razor,2004,1,3,"0491402,0249046,0004898,0000494,0607185,081798..."
99,412142,606030,Maternity,2004,1,4,"0491402,0249046,0004898,0000494,0607185,081798..."
100,412142,606018,Damned If You Do,2004,1,5,"0491402,0249046,0004898,0000494,0607185,081798..."


In [7]:
episode = episode_df.iloc[0]
episode

series_id                                                412142
episode_id                                               606035
title                                                     Pilot
year                                                       2004
season                                                        1
episode                                                       1
cast          0491402,0249046,0004898,0000494,0607185,081798...
Name: 96, dtype: object

In [10]:
series = ia.get_movie('0412142')
possible_cast = cast_from_series('0412142', count=2)
print(len(possible_cast))

123


In [11]:
temp = [x for x in episode['cast'].split(',') if x in possible_cast]
print(len(temp))

7


In [13]:
data = cast_from_episode(episode['episode_id'], cast=possible_cast)
data

Getting cast: 100%|█████████████████████████████| 7/7 [00:17<00:00,  2.53s/it]


[{'name': 'Dr. Gregory House', 'personID': '0491402'},
 {'name': 'Dr. Lisa Cuddy', 'personID': '0249046'},
 {'name': 'Dr. Eric Foreman', 'personID': '0004898'},
 {'name': 'Dr. James Wilson', 'personID': '0000494'},
 {'name': 'Dr. Allison Cameron', 'personID': '0607185'},
 {'name': 'Dr. Robert Chase', 'personID': '0817980'}]

In [14]:
cast_ids = []
for cast in episode_df['cast']:
    cast_ids.extend(cast.split(','))
cast_ids = list(set(cast_ids))

In [15]:
data = []
for cast_id in cast_ids:
    temp = episode_df[episode_df['cast'].str.contains(cast_id)]
    datum = {'cast_id': cast_id,
             'count': temp.shape[0]}
    data.append(datum)
cast_count = pd.DataFrame(data)
cast_count = cast_count.sort_values(by='count')
cast_count.head()

,cast_id,count
830,0286022,1
1085,0086883,1
1084,0100762,1
1083,1634944,1
1082,0454539,1


In [16]:
db = './data/headshots/house_2004_0412142/Hugh Laurie'
files = [x for x in Path('./data/clustering/house_2004_0412142/chinese_whisper/0').iterdir() if x.parts[-1] != '.ipynb']

In [23]:
df = DeepFace.find(str(files[0]), db_path=str(db), enforce_detection=False, silent=True)
df[0]

,identity,source_x,source_y,source_w,source_h,VGG-Face_cosine
0,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.164054
1,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.183883
2,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.247532
3,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.270570
4,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.312883
5,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.325792
6,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.345140
7,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.367123
8,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.368306
9,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.378863


In [19]:
df[0]

,identity,source_x,source_y,source_w,source_h,VGG-Face_cosine
0,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.164054
1,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.183883
2,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.247532
3,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.270570
4,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.312883
5,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.325792
6,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.345140
7,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.367123
8,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.368306
9,data/headshots/house_2004_0412142/Hugh Laurie/...,0,0,142,142,0.378863
